In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [7]:
import tensorflow as tf
import keras 
from keras.applications import DenseNet201
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, NumpyArrayIterator
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import confusion_matrix
from keras import models, layers, optimizers
import tensorflow.keras.backend as K
import os

In [8]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"
gpus = tf.config.list_physical_devices('GPU'); print(gpus)
if len(gpus)==1: strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
else: strategy = tf.distribute.MirroredStrategy()

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [9]:
tf.config.optimizer.set_experimental_options({"auto_mixed_precision": True})
print('Mixed precision enabled')

Mixed precision enabled


In [10]:
conv_base = DenseNet201(weights='imagenet', include_top = False, input_shape=(300,300,3))

74842112/74836368 [==============================] - 2s 0us/step


In [11]:
X = np.load("../input/pcb-image-dataset1/xtrain.npy")
y = np.load("../input/pcb-image-dataset1/ytrain.npy")

In [12]:
datagen = ImageDataGenerator(
    featurewise_center = True,
    featurewise_std_normalization = True
)

datagen.fit(X)

In [13]:
nos = X.shape[0]
bs = int(nos/8)

numit = NumpyArrayIterator(
    X, y, datagen, batch_size=bs, shuffle=False, sample_weight=None,
    seed=None, data_format=None, save_to_dir=None, save_prefix='',
    save_format='png', subset=None, dtype=None
)

samples = np.zeros(shape=(nos,9,9,1920))
labels = np.zeros(shape=(nos))
i = 0
for samples_batch,labels_batch in numit:
    f = conv_base.predict(samples_batch)
    print(f.shape)
    samples[i*bs:(i+1)*bs] = f
    labels[i*bs:(i+1)*bs] = labels_batch
    i+=1
    if (i*bs)>=nos:
        break

(59, 9, 9, 1920)
(59, 9, 9, 1920)
(59, 9, 9, 1920)
(59, 9, 9, 1920)
(59, 9, 9, 1920)
(59, 9, 9, 1920)
(59, 9, 9, 1920)
(59, 9, 9, 1920)


In [14]:
samples = samples.reshape(nos,9*9*1920)

In [15]:
del X
del y

In [16]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=8)

In [17]:
pbounds = {
    'learning_rate':(1e-4, 1e-2)
}

In [28]:
def evaluate_network(learning_rate):
    f = 1
    accuracies = []
    for train_index, test_index in skf.split(samples, labels):
        print('Fold {}'.format(f))
        X_train = samples[train_index]
        X_test = samples[test_index]
        y_train = labels[train_index]
        y_test = labels[test_index]

        K.clear_session()
        with strategy.scope():
            model = models.Sequential()
            model.add(layers.Dense(256, activation = 'relu', input_dim = 9*9*1920))
            model.add(layers.Dropout(0.5))
            model.add(layers.Dense(1, activation = 'sigmoid'))
            model.compile(optimizer = optimizers.Adam(lr = learning_rate),loss = 'binary_crossentropy',metrics = ['accuracy'])
            my_callbacks = [EarlyStopping(monitor = 'val_loss', patience=5, restore_best_weights=True)]
            history = model.fit(X_train, y_train, epochs = 100, batch_size = 8, validation_data = (X_test, y_test), shuffle = True, callbacks = my_callbacks)
            
    
        test_preds = model.predict(X_test, batch_size = 20)
        test_predclass = np.array([1.0 if i>0.5 else 0.0 for i in test_preds])

        accuracies.append([confusion_matrix(y_test,test_predclass)[0][0]/(y_test == 0.0).sum(), confusion_matrix(y_test,test_predclass)[1][1]/(y_test == 1.0).sum()])
        f+=1


    zero_acc = 0
    one_acc = 0

    for i in accuracies:
        zero_acc+= i[0]
        one_acc+=i[1]

    zero_acc/=8
    one_acc/=8

    print("Accuracies: ",accuracies)
    return (zero_acc+one_acc)

In [51]:
# !pip install bayesian-optimization

In [20]:
from bayes_opt import BayesianOptimization

In [29]:
optimizer = BayesianOptimization(
    f = evaluate_network,
    pbounds = pbounds,
    verbose = 2
)

In [30]:
optimizer.maximize(
    init_points=1,
    n_iter=3,
)

|   iter    |  target   | learni... |
-------------------------------------
Fold 1
Epoch 1/100
14/14 [==============================] - 1s 43ms/step - loss: 63.2958 - accuracy: 0.6586 - val_loss: 32.4236 - val_accuracy: 0.7627
Epoch 2/100
14/14 [==============================] - 1s 37ms/step - loss: 16.1752 - accuracy: 0.8208 - val_loss: 18.1731 - val_accuracy: 0.7966
Epoch 3/100
14/14 [==============================] - 1s 36ms/step - loss: 6.8575 - accuracy: 0.8402 - val_loss: 8.2056 - val_accuracy: 0.7288
Epoch 4/100
14/14 [==============================] - 1s 37ms/step - loss: 2.0360 - accuracy: 0.9298 - val_loss: 4.8086 - val_accuracy: 0.7966
Epoch 5/100
14/14 [==============================] - 1s 38ms/step - loss: 0.8069 - accuracy: 0.9637 - val_loss: 2.6777 - val_accuracy: 0.8305
Epoch 6/100
14/14 [==============================] - 1s 38ms/step - loss: 0.5326 - accuracy: 0.9443 - val_loss: 2.4301 - val_accuracy: 0.7797
Epoch 7/100
14/14 [==============================] - 0s 26ms/

Fold 4
Epoch 1/100
14/14 [==============================] - 1s 42ms/step - loss: 75.7814 - accuracy: 0.6610 - val_loss: 13.4382 - val_accuracy: 0.7627
Epoch 2/100
14/14 [==============================] - 1s 36ms/step - loss: 10.7965 - accuracy: 0.8160 - val_loss: 4.4088 - val_accuracy: 0.7458
Epoch 3/100
14/14 [==============================] - 1s 36ms/step - loss: 1.8788 - accuracy: 0.8838 - val_loss: 3.1679 - val_accuracy: 0.7458
Epoch 4/100
14/14 [==============================] - 1s 37ms/step - loss: 0.9811 - accuracy: 0.8959 - val_loss: 1.8805 - val_accuracy: 0.8136
Epoch 5/100
14/14 [==============================] - 0s 34ms/step - loss: 0.5978 - accuracy: 0.9177 - val_loss: 1.7629 - val_accuracy: 0.7627
Epoch 6/100
14/14 [==============================] - 1s 66ms/step - loss: 0.3532 - accuracy: 0.9177 - val_loss: 1.2621 - val_accuracy: 0.7627
Epoch 7/100
14/14 [==============================] - 1s 37ms/step - loss: 0.2928 - accuracy: 0.9322 - val_loss: 1.2409 - val_accuracy: 0.7

14/14 [==============================] - 0s 35ms/step - loss: 0.2555 - accuracy: 0.9734 - val_loss: 1.0765 - val_accuracy: 0.8644
Epoch 17/100
14/14 [==============================] - 1s 37ms/step - loss: 0.3529 - accuracy: 0.9782 - val_loss: 0.9065 - val_accuracy: 0.8814
Epoch 18/100
14/14 [==============================] - 1s 38ms/step - loss: 0.0932 - accuracy: 0.9831 - val_loss: 0.8448 - val_accuracy: 0.8475
Epoch 19/100
14/14 [==============================] - 0s 27ms/step - loss: 0.0996 - accuracy: 0.9831 - val_loss: 0.8770 - val_accuracy: 0.8814
Epoch 20/100
14/14 [==============================] - 0s 24ms/step - loss: 0.0849 - accuracy: 0.9806 - val_loss: 0.9556 - val_accuracy: 0.8475
Epoch 21/100
14/14 [==============================] - 0s 36ms/step - loss: 0.0598 - accuracy: 0.9855 - val_loss: 0.8206 - val_accuracy: 0.8644
Epoch 22/100
14/14 [==============================] - 1s 37ms/step - loss: 0.1111 - accuracy: 0.9903 - val_loss: 0.7373 - val_accuracy: 0.8644
Epoch 23/100

ValueError: array must not contain infs or NaNs

In [31]:
optimizer.max

{'target': 1.5892241379310346,
 'params': {'learning_rate': 0.007152386412153171}}

In [ ]:
optimizer.max['params']['learning_rate']

In [36]:
xtest = np.load('../input/pcb-image-dataset1/xtest.npy')
ytest = np.load('../input/pcb-image-dataset1/ytest.npy')


In [37]:
datagentest = ImageDataGenerator(
    featurewise_center = True,
    featurewise_std_normalization = True
)
datagentest.fit(xtest)

In [38]:
testnos = ytest.shape[0]

numittest = NumpyArrayIterator(
    xtest, ytest, datagentest, batch_size=23, shuffle=False, sample_weight=None,
    seed=None, data_format=None, save_to_dir=None, save_prefix='',
    save_format='png', subset=None, dtype=None
)

i = 0
test_samples = np.zeros(shape=(testnos,9,9,1920))
test_labels = np.zeros(shape=(testnos))
for samples_batch,labels_batch in numittest:
    g = conv_base.predict(samples_batch)
    print(g.shape, i)
    i+=1
    test_samples[(i*23):((i+1)*23)] = g
    test_labels[(i*23):((i+1)*23)] = labels_batch
    if(i==23):
        break

(23, 9, 9, 1920) 0
(23, 9, 9, 1920) 1
(23, 9, 9, 1920) 2
(23, 9, 9, 1920) 3
(23, 9, 9, 1920) 4
(23, 9, 9, 1920) 5
(23, 9, 9, 1920) 6
(23, 9, 9, 1920) 7
(23, 9, 9, 1920) 8
(23, 9, 9, 1920) 9
(23, 9, 9, 1920) 10
(23, 9, 9, 1920) 11
(23, 9, 9, 1920) 12
(23, 9, 9, 1920) 13
(23, 9, 9, 1920) 14
(23, 9, 9, 1920) 15
(23, 9, 9, 1920) 16
(23, 9, 9, 1920) 17
(23, 9, 9, 1920) 18
(23, 9, 9, 1920) 19
(23, 9, 9, 1920) 20
(23, 9, 9, 1920) 21
(23, 9, 9, 1920) 22


In [39]:
test_samples = test_samples.reshape(testnos,9*9*1920)

In [40]:
test_samples.shape, test_labels.shape

((552, 155520), (552,))

In [41]:
del xtest;del ytest;del numittest;del numit;

In [ ]:
del model
K.clear_session()

In [42]:
with strategy.scope():
    model = models.Sequential()
    model.add(layers.Dense(256, activation = 'relu', dtype='float32', input_dim = 9*9*1920))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(1, dtype='float32',activation = 'sigmoid'))
    model.compile(optimizer = optimizers.Adam(lr = optimizer.max['params']['learning_rate']),loss = 'binary_crossentropy',metrics = ['accuracy'])
    my_callbacks = [EarlyStopping(monitor = 'val_loss', patience=5, restore_best_weights=True)]
    historytest = model.fit(samples, labels, epochs = 100, batch_size = 8, validation_data = (test_samples, test_labels), shuffle = True,  callbacks = my_callbacks)

Epoch 1/100
8/8 [==============================] - 2s 214ms/step - loss: 134.0584 - accuracy: 0.5784 - val_loss: 97.8387 - val_accuracy: 0.3551
Epoch 2/100
8/8 [==============================] - 2s 201ms/step - loss: 34.1715 - accuracy: 0.7415 - val_loss: 67.7797 - val_accuracy: 0.4638
Epoch 3/100
8/8 [==============================] - 2s 209ms/step - loss: 17.4663 - accuracy: 0.8178 - val_loss: 9.4761 - val_accuracy: 0.8750
Epoch 4/100
8/8 [==============================] - 2s 304ms/step - loss: 5.0977 - accuracy: 0.8347 - val_loss: 4.9555 - val_accuracy: 0.7283
Epoch 5/100
8/8 [==============================] - 2s 229ms/step - loss: 1.2173 - accuracy: 0.8602 - val_loss: 3.9534 - val_accuracy: 0.6504
Epoch 6/100
8/8 [==============================] - 2s 212ms/step - loss: 0.6202 - accuracy: 0.8962 - val_loss: 2.2166 - val_accuracy: 0.7736
Epoch 7/100
8/8 [==============================] - 2s 216ms/step - loss: 0.4921 - accuracy: 0.8877 - val_loss: 1.9762 - val_accuracy: 0.7409
Epoch 8

In [43]:
test_preds = model.predict(test_samples, batch_size = 20)

In [44]:
test_predclass = np.array([1.0 if i>0.5 else 0.0 for i in test_preds])

In [45]:
confusion_matrix(test_labels,test_predclass)[0][0]/(test_labels == 0.0).sum(), confusion_matrix(test_labels,test_predclass)[1][1]/(test_labels == 1.0).sum()

(0.5, 0.8212765957446808)